In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Read Inside Airbnb data") \
    .getOrCreate()

In [4]:
listings = spark.read.csv("../data/listings.csv.gz", 
    header=True,
    inferSchema=True,
    sep=",", 
    quote='"',
    escape='"', 
    multiLine=True,
    mode="PERMISSIVE" 
)

In [5]:
listings.printSchema()

root
 |-- id: long (nullable = true)
 |-- listing_url: string (nullable = true)
 |-- scrape_id: long (nullable = true)
 |-- last_scraped: date (nullable = true)
 |-- source: string (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- neighborhood_overview: string (nullable = true)
 |-- picture_url: string (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- host_url: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- host_since: date (nullable = true)
 |-- host_location: string (nullable = true)
 |-- host_about: string (nullable = true)
 |-- host_response_time: string (nullable = true)
 |-- host_response_rate: string (nullable = true)
 |-- host_acceptance_rate: string (nullable = true)
 |-- host_is_superhost: string (nullable = true)
 |-- host_thumbnail_url: string (nullable = true)
 |-- host_picture_url: string (nullable = true)
 |-- host_neighbourhood: string (nullable = true)
 |-- host_listings_count: int

In [28]:
# 1. Get a non-null picture URL for any property ("picture_url" field)
# Select any non-null picture URL
from pyspark.sql.functions import col

listings \
    .select(col('picture_url')) \
    .dropna() \
    .limit(1) \
    .show(20, truncate = False)

+------------------------------------------------------------------------------------------------------+
|picture_url                                                                                           |
+------------------------------------------------------------------------------------------------------+
|https://a0.muscache.com/pictures/miso/Hosting-13913/original/d755aa6d-cebb-4464-80be-2722c921e8d5.jpeg|
+------------------------------------------------------------------------------------------------------+



In [29]:
# 2. Get number of properties that get more than 10 reviews per month
listings \
    .filter(col('reviews_per_month') > 10) \
    .count()

66

In [30]:
# 3. Get properties that have more bathrooms than bedrooms
listings \
    .filter(col('bathrooms') > col('bedrooms')) \
    .select('name', 'bathrooms', 'bedrooms') \
    .show(10, truncate = False)

+----------------------------------------------+---------+--------+
|name                                          |bathrooms|bedrooms|
+----------------------------------------------+---------+--------+
|Cosy Double studio in Zone 2 Hammersmith (1)  |1.5      |1       |
|LONDON DETACHED HOUSE*ElecGates etc           |2.0      |1       |
|Designer room Park Views 4 mins zone 1 station|1.5      |1       |
|Maisonette in Central London Zone 1           |1.5      |1       |
|West London,loft ensuite, 5min2tube           |1.5      |1       |
|Shoreditch Loft                               |1.5      |1       |
|Five minute walk to South Bank                |1.5      |1       |
|Stunning double room own bathroom             |4.0      |1       |
|Also five minutes to South Bank               |1.5      |1       |
|Studio 20 min from center                     |1.0      |0       |
+----------------------------------------------+---------+--------+
only showing top 10 rows


In [31]:
# 4. Get properties where the price is greater than 5,000. Collect the result as a Python list
# Remember to convert a price into a number first!
from pyspark.sql.functions import regexp_replace

listings_with_price = listings \
  .withColumn('price_numeric', regexp_replace('price', '[$,]', '').cast('float'))

listings_with_price \
    .filter(col('price_numeric') > 5000) \
    .select('name', 'price') \
    .collect()
    

[Row(name='Room in a cosy flat. Central, clean', price='$8,000.00'),
 Row(name='Spacious Private Ground Floor Room', price='$6,309.00'),
 Row(name='No Longer Available', price='$53,588.00'),
 Row(name='Bright & airy DoubleBed with EnSuite in Zone 2!', price='$74,100.00'),
 Row(name='The Apartments by The Sloane Club, Two Bedroom Apt', price='$7,377.00'),
 Row(name='The Apartments by The Sloane Club, L 2 Bedroom Apt', price='$7,377.00'),
 Row(name='Single room. 7ft x 9ft - Over looking garden', price='$6,523.00'),
 Row(name='Close To London Eye (TUR)', price='$6,666.00'),
 Row(name='Beautiful 2 BR flat in Kilburn with free parking', price='$6,000.00'),
 Row(name='Semi-detached mews house in Knightsbridge.', price='$7,019.00'),
 Row(name='Affordable Spacious  Room on the edge of the city', price='$6,000.00'),
 Row(name='Henry’s Townhouse, London', price='$6,500.00'),
 Row(name='City Suite', price='$5,353.00'),
 Row(name='Hyde Park Suite', price='$5,653.00'),
 Row(name='SHORT WALK TO LOND

In [27]:
# 5. Get a list of properties with the following characteristics:
# * price < 150
# * more than 20 reviews
# * review_scores_rating > 4.5
# Consider using the "&" operator
listings_with_price \
    .filter((col('price_numeric') < 150) & (col('number_of_reviews') > 20) & (col('review_scores_rating') > 4.5)) \
    .select('name', 'price', 'number_of_reviews', 'review_scores_rating') \
    .collect()

[Row(name='Holiday London DB Room Let-on going', price='$70.00', number_of_reviews=55, review_scores_rating=4.85),
 Row(name='Bright Chelsea  Apartment. Chelsea!', price='$149.00', number_of_reviews=97, review_scores_rating=4.8),
 Row(name='You are GUARANTEED to love this', price='$90.00', number_of_reviews=730, review_scores_rating=4.87),
 Row(name='SUNNY ROOM PRIVATE BATHROOM PLUS BREAKFAST', price='$61.00', number_of_reviews=387, review_scores_rating=4.77),
 Row(name='SPACIOUS ROOM IN CONTEMPORARY STYLE FLAT', price='$49.00', number_of_reviews=72, review_scores_rating=4.97),
 Row(name='Room with a view, shared flat,  central  Bankside', price='$96.00', number_of_reviews=137, review_scores_rating=4.7),
 Row(name='You Will Save Money Here', price='$71.00', number_of_reviews=639, review_scores_rating=4.89),
 Row(name='Quiet Comfortable Room in Fulham', price='$48.00', number_of_reviews=266, review_scores_rating=4.74),
 Row(name='Pleasant Single Room in zone 1.', price='$50.00', number_

In [25]:
# 6. Get a list of properties with the following characteristics:
# * price < 150 OR more than one bathroom
# Use the "|" operator to implement the OR operator
listings_with_price \
    .filter((col('price_numeric') < 150) | (col('bathrooms') > 1)) \
    .select('name', 'price', 'bathrooms') \
    .collect()

[Row(name='Holiday London DB Room Let-on going', price='$70.00', bathrooms=1.0),
 Row(name='Bright Chelsea  Apartment. Chelsea!', price='$149.00', bathrooms=1.0),
 Row(name='Very Central Modern 3-Bed/2 Bath By Oxford St W1', price='$411.00', bathrooms=2.0),
 Row(name='Kew Gardens 3BR house in cul-de-sac', price='$280.00', bathrooms=1.5),
 Row(name='You are GUARANTEED to love this', price='$90.00', bathrooms=0.0),
 Row(name='SUNNY ROOM PRIVATE BATHROOM PLUS BREAKFAST', price='$61.00', bathrooms=1.0),
 Row(name='Short Term Home', price='$340.00', bathrooms=2.0),
 Row(name='SPACIOUS ROOM IN CONTEMPORARY STYLE FLAT', price='$49.00', bathrooms=1.0),
 Row(name='Room with a view, shared flat,  central  Bankside', price='$96.00', bathrooms=1.0),
 Row(name='You Will Save Money Here', price='$71.00', bathrooms=1.0),
 Row(name='Quiet Comfortable Room in Fulham', price='$48.00', bathrooms=1.0),
 Row(name='Room with a garden', price='$76.00', bathrooms=1.0),
 Row(name='Pleasant Single Room in zone 

In [32]:
# 7. Get the highest listing price in this dataset
# Consider using the "max" function from "pyspark.sql.functions"
from pyspark.sql.functions import max

listings_with_price \
    .select(max(col('price_numeric'))) \
    .show()

+------------------+
|max(price_numeric)|
+------------------+
|         1085147.0|
+------------------+



In [ ]:
# 8. Get the name and a price of property with the highest price
# Try to use "collect" method to get the highest price first, and then use it in a "filter" call 
from pyspark.sql.functions import max

res = listings_with_price \
    .select(max('price_numeric').alias('max_price')) \
    .collect()

max_price = res[0]['max_price']

listings_with_price \
    .filter(col('price_numeric') == max_price) \
    .select('name', 'price_numeric') \
    .show()

AnalysisException: [MISSING_GROUP_BY] The query does not include a GROUP BY clause. Add GROUP BY or turn it into the window functions using OVER clauses. SQLSTATE: 42803;
Aggregate [name#5, max(price_numeric#339) AS max(price_numeric)#354]
+- Project [id#0L, listing_url#1, scrape_id#2L, last_scraped#3, source#4, name#5, description#6, neighborhood_overview#7, picture_url#8, host_id#9, host_url#10, host_name#11, host_since#12, host_location#13, host_about#14, host_response_time#15, host_response_rate#16, host_acceptance_rate#17, host_is_superhost#18, host_thumbnail_url#19, host_picture_url#20, host_neighbourhood#21, host_listings_count#22, host_total_listings_count#23, host_verifications#24, ... 55 more fields]
   +- Relation [id#0L,listing_url#1,scrape_id#2L,last_scraped#3,source#4,name#5,description#6,neighborhood_overview#7,picture_url#8,host_id#9,host_url#10,host_name#11,host_since#12,host_location#13,host_about#14,host_response_time#15,host_response_rate#16,host_acceptance_rate#17,

In [15]:
# 9. Get the number of hosts in the dataset


In [16]:
# 10. Get listings with a first review in 2024
# Consider using the "year" function from "pyspark.sql.functions"
